## Electroral Bond Project:

* This is an analysis of the data released by the ECI dated 14 March and contains record from 12 April 2019 to 24 Jan 2024

#### Formating the data from CSV to Pandas

In [1]:
import pandas as pd

In [2]:
donor_data = pd.read_csv("donor.csv").dropna()

In [3]:
party_data = pd.read_csv("party.csv").dropna()

In [4]:
party_data

,Date of\nEncashment,Name of the Political Party,Denomination
0,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0
1,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0
2,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,10000000.0
3,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0
4,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0
...,...,...,...
20841,24/Jan/2024,JANASENA PARTY,1000000.0
20842,24/Jan/2024,JANASENA PARTY,1000000.0
20843,24/Jan/2024,JANASENA PARTY,1000000.0
20844,24/Jan/2024,JANASENA PARTY,1000000.0


In [5]:
donor_data

,Date of Purchase,Purchaser Name,Denomination
0,12/Apr/2019,A B C INDIA LIMITED,100000.0
1,12/Apr/2019,A B C INDIA LIMITED,100000.0
2,12/Apr/2019,A B C INDIA LIMITED,1000000.0
3,12/Apr/2019,A B C INDIA LIMITED,1000000.0
4,12/Apr/2019,A B C INDIA LIMITED,100000.0
...,...,...,...
19202,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000.0
19203,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000.0
19204,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000.0
19205,11/Jan/2024,WESTWELL GASES PRIVATE LIMITED,10000000.0


* Here we can look at the raw data that we have read from the CSV file 


#### Data Cleaning (entries added up)

* This part reduces the original data to a compressed format with adjacent rows merged

In [6]:
len(party_data["Date of\nEncashment"].keys()),len(donor_data["Date of Purchase"].keys())

(20421, 18871)

In [7]:
bad_chars = ['(',')']

## Party Data 

In [45]:
l=[]
prev = None
count = 0
s = 0
idx = 0
for j in range(len(party_data["Date of\nEncashment"].keys())):
    i= party_data["Date of\nEncashment"].keys()[j]
#     print(donor_data["Date of Purchase"][i])
    key = party_data["Date of\nEncashment"][i][:7] + party_data["Date of\nEncashment"][i][9:] +" " +party_data["Name of the Political Party"][i]
    if(prev == None):
        prev = key
        count = 1
        s = int(party_data["Denomination"][i])
    elif(prev == key):
        count+=1
        s = s + int(party_data["Denomination"][i])
    else:
        name = ''.join((filter(lambda i: i not in bad_chars, prev[9:])))
        l.append([idx,prev[:9],name,count,s])
        idx+=1
        prev = key
        count = 1
        s = int(party_data["Denomination"][i])
    
name = ''.join((filter(lambda i: i not in bad_chars, prev[9:])))
l.append([idx,prev[:9],name,count,s])
        
        
        

In [46]:
df_party = pd.DataFrame(l, columns=['index','Date', 'party',"count","denomination"])

In [47]:
l=[]
prev = None
count = 0
s = 0
idx = 0
for j in range(len(donor_data["Date of Purchase"].keys())):
    i= donor_data["Date of Purchase"].keys()[j]
#     print(donor_data["Date of Purchase"][i])
    key = donor_data["Date of Purchase"][i][:7] + donor_data["Date of Purchase"][i][9:] +" " +donor_data["Purchaser Name"][i]
    if(prev == None):
        prev = key
        count = 1
        s = int(donor_data["Denomination"][i])
    elif(prev == key):
        count+=1
        s = s + int(donor_data["Denomination"][i])
    else:
        name = ''.join((filter(lambda i: i not in bad_chars, prev[9:])))
        l.append([idx,prev[:9],name,count,s])
        idx+=1
        prev = key
        count = 1
        s = int(donor_data["Denomination"][i])
    
name = ''.join((filter(lambda i: i not in bad_chars, prev[9:])))
l.append([idx,prev[:9],name,count,s])
        
        


In [122]:
df_donor = pd.DataFrame(l, columns=['index','Date', 'donor',"count","denomination"])

In [123]:
df_party['date'] = pd.to_datetime(df_party['Date']).dt.normalize()
df_donor['date'] = pd.to_datetime(df_donor['Date']).dt.normalize()



from datetime import timedelta
df_donor['expiredate'] = df_donor['date'] + timedelta(days = 15)


df_party['funds'] = df_party['denomination']/10000000
df_donor['donation'] = df_donor['denomination']/10000000

#### After Formating

* After formating we can see we are left with 

In [124]:
print("unique donation",len(df_donor)," unique encashment",len(df_party))

unique donation 1980  unique encashment 513


In [125]:
df_party

,index,Date,party,count,denomination,date,funds
0,0,12/Apr/19,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,33,60000000,2019-04-12,6.0
1,1,12/Apr/19,BHARAT RASHTRA SAMITHI,27,225000000,2019-04-12,22.5
2,2,12/Apr/19,BHARTIYA JANTA PARTY,402,2463000000,2019-04-12,246.3
3,3,12/Apr/19,"PRESIDENT, ALL INDIA CONGRESS COMMITTEE",20,101000000,2019-04-12,10.1
4,4,12/Apr/19,SHIVSENA POLITICAL PARTY,19,37000000,2019-04-12,3.7
...,...,...,...,...,...,...,...
508,508,22/Jan/24,"PRESIDENT, ALL INDIA CONGRESS COMMITTEE",10,10000000,2024-01-22,1.0
509,509,22/Jan/24,TELUGU DESAM PARTY,3,30000000,2024-01-22,3.0
510,510,23/Jan/24,DMK PARTY IN PARLIAMENT,20,200000000,2024-01-23,20.0
511,511,24/Jan/24,BHARTIYA JANTA PARTY,7,70000000,2024-01-24,7.0


In [126]:
df_donor

,index,Date,donor,count,denomination,date,expiredate,donation
0,0,12/Apr/19,A B C INDIA LIMITED,13,4000000,2019-04-12,2019-04-27,0.40
1,1,12/Apr/19,ACROPOLIS MAINTENANCE SERVICES PRIVATE LIMITED,7,2500000,2019-04-12,2019-04-27,0.25
2,2,12/Apr/19,ARIHANT ENTERPRISES,4,4000000,2019-04-12,2019-04-27,0.40
3,3,12/Apr/19,CHOUDHARY GARMENTS,2,2000000,2019-04-12,2019-04-27,0.20
4,4,12/Apr/19,ESSEL MINING AND INDS LTD,59,500000000,2019-04-12,2019-04-27,50.00
...,...,...,...,...,...,...,...,...
1975,1975,11/Jan/24,VEDIKA VANIJYA PVT LTD-SELF A/C,5,50000000,2024-01-11,2024-01-26,5.00
1976,1976,11/Jan/24,VIDUR GUPTA,5,5000000,2024-01-11,2024-01-26,0.50
1977,1977,11/Jan/24,VIHAAN AUTO VENTURES PRIVATE LIMITED,2,20000000,2024-01-11,2024-01-26,2.00
1978,1978,11/Jan/24,WESTERN UP POWER TRANSMISSION COMPANY LI MITED,30,300000000,2024-01-11,2024-01-26,30.00


* Now we can easily search for text query on donor column
* As well as fund amount query

In [127]:
df_donor[df_donor['donor'].str.contains("TORRENT")]

,index,Date,donor,count,denomination,date,expiredate,donation
180,180,07/May/19,TORRENT POWER LIMITED,12,75000000,2019-05-07,2019-05-22,7.5
181,181,07/May/19,TORRENT PHARMACEUTICALS LIMITED,7,25000000,2019-05-07,2019-05-22,2.5
245,245,10/May/19,TORRENT PHARMACEUTICALS LIMITED,5,50000000,2019-05-10,2019-05-25,5.0
273,273,01/Oct/19,TORRENT POWER LIMITED,7,70000000,2019-10-01,2019-10-16,7.0
274,274,01/Oct/19,TORRENT PHARMACEUTICALS LIMITED,5,50000000,2019-10-01,2019-10-16,5.0
280,280,03/Oct/19,TORRENT POWER LIMITED,3,30000000,2019-10-03,2019-10-18,3.0
473,473,07/Apr/21,TORRENT POWER LIMITED,12,75000000,2021-04-07,2021-04-22,7.5
474,474,07/Apr/21,TORRENT PHARMACEUTICALS LIMITED,12,75000000,2021-04-07,2021-04-22,7.5
729,729,03/Jan/22,TORRENT POWER LTD,3,30000000,2022-01-03,2022-01-18,3.0
803,803,07/Jan/22,TORRENT PHARMACEUTICALS LTD,10,100000000,2022-01-07,2022-01-22,10.0


In [128]:
sum(df_donor[df_donor['donor'].str.contains("TORRENT")]['donation'])

184.0

## Search Begins

* In this part we have defined a cell to isolate the cleaned data from plotting data so just put in your pandas dataframe query and let the plotting code run 

In [129]:
import plotly.express as px

In [130]:
plt_data_party = df_party[df_party['party'].str.contains("RASHTRA")]
plt_data_donor = df_donor[df_donor['donor'].str.contains("FUTURE")]

In [131]:
# df_party['Date'].str.contains("Jan/22") & 

In [132]:
df_party[df_party['funds']>200]

,index,Date,party,count,denomination,date,funds
2,2,12/Apr/19,BHARTIYA JANTA PARTY,402,2463000000,2019-04-12,246.30
23,23,18/Apr/19,BHARTIYA JANTA PARTY,402,2275800000,2019-04-18,227.58
29,29,20/Apr/19,BHARTIYA JANTA PARTY,420,2107500000,2019-04-20,210.75
63,63,10/May/19,BHARTIYA JANTA PARTY,492,3961500000,2019-05-10,396.15
69,69,14/May/19,BHARTIYA JANTA PARTY,257,2412500000,2019-05-14,241.25
192,192,10/Jan/22,BHARTIYA JANTA PARTY,267,2512500000,2022-01-10,251.25
232,232,12/Apr/22,BHARAT RASHTRA SAMITHI,268,2680000000,2022-04-12,268.00
290,290,16/Nov/22,BHARTIYA JANTA PARTY,213,2017500000,2022-11-16,201.75
380,380,13/Jul/23,BHARAT RASHTRA SAMITHI,332,2240000000,2023-07-13,224.00


In [133]:
df_donor[df_donor['donation']>50]

,index,Date,donor,count,denomination,date,expiredate,donation
56,56,16/Apr/19,KEVENTER FOODPARK INFRA LIMITED,55,550000000,2019-04-16,2019-05-01,55.0
175,175,07/May/19,KEVENTER FOODPARK INFRA LIMITED,84,750000000,2019-05-07,2019-05-22,75.0
193,193,08/May/19,KEVENTER FOODPARK INFRA LIMITED,65,650000000,2019-05-08,2019-05-23,65.0
196,196,08/May/19,MADANLAL LTD.,110,1100000000,2019-05-08,2019-05-23,110.0
232,232,10/May/19,MADANLAL LTD.,89,755000000,2019-05-10,2019-05-25,75.5
395,395,21/Oct/20,FUTURE GAMING AND HOTEL SERVICES PR,60,600000000,2020-10-21,2020-11-05,60.0
405,405,27/Oct/20,FUTURE GAMING AND HOTEL SERVICES PR,90,900000000,2020-10-27,2020-11-11,90.0
446,446,05/Apr/21,ESSEL MINING AND INDS LTD,59,545000000,2021-04-05,2021-04-20,54.5
505,505,09/Apr/21,FUTURE GAMING AND HOTEL SERVICES PR,60,600000000,2021-04-09,2021-04-24,60.0
606,606,05/Oct/21,FUTURE GAMING AND HOTEL SERVICES PR,195,1950000000,2021-10-05,2021-10-20,195.0


In [134]:
sum(df_donor[df_donor['Date'].str.contains("Jul/23")]['donation'])

812.8002000000001

In [135]:
sum(df_party[df_party['Date'].str.contains("Jul/23")]['funds'])

812.7501000000001

In [136]:
sum(df_party[df_party['party'].str.contains("BHARTIYA") & df_party['Date'].str.contains("Jul/23")]['funds'])

86.9301

In [137]:
df_party['party'] = df_party['party'].astype('str') 

In [142]:
plt_data_party = df_party[df_party['Date'].str.contains("/23")]
plt_data_donor = df_donor[df_donor['Date'].str.contains("/23")]

In [143]:
import plotly.express as px
import pandas as pd


fig = px.timeline(plt_data_donor, x_start="date", x_end="expiredate", y="donation", color="donor")

scatter_fig = px.scatter(plt_data_party, x='date', y='funds',color='party',size = 'funds')

for trace in scatter_fig.data:
    fig.add_trace(trace)

fig.show()

In [85]:
import plotly.express as px
import plotly as plt

# Assuming df_party and df_donor are your DataFrames
# Replace 'date', 'funds', 'party', 'donor', and 'donation' with the actual column names in your DataFrames

# Create bar plot for df_party

bar_fig = px.bar(plt_data_party, x='date', y='funds', color='party',
                 labels={'date': 'Date', 'funds': 'Funds', 'party': 'Party'},
                 title='Funds Raised by Party Over Time',log_y = True)

# bar_fig.update_xaxes(tick0 = df_party['date'][0],dtick = 60)
# bar_fig.update(dtick = 0)
# Create scatter plot for df_donor
scatter_fig = px.scatter(plt_data_donor, x='date', y='donation',color='donor',
                         labels={'date': 'Date', 'donation': 'Donation', 'donor': 'Donor'},
                         title='Donations Over Time by Donor',size = 'donation',log_y = True)

# Create a combined figure
combined_fig = plt.graph_objects.Figure()

# Add traces from the bar plot
for trace in bar_fig.data:
    combined_fig.add_trace(trace)

    
# Add traces from the scatter plot
for trace in scatter_fig.data:
    combined_fig.add_trace(trace)

    
# Show the combined plot
# , xaxis_range=[start_date, end_date]
combined_fig.update_layout(barmode='group',bargap = 0.3, autosize=True)
combined_fig.update_layout(plot_bgcolor='#64706e')

combined_fig.show()
